In [13]:
# Baixando Keras Tuner
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [2]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Carregar o dataset Titanic
titanic = pd.read_csv('https://raw.githubusercontent.com/EdgarNoda/INTI6-2024/main/datasets/Titanic/train.csv')

# Pré-processamento
titanic['Sex'] = titanic['Sex'].map({'male': 0, 'female': 1})
titanic['Embarked'] = titanic['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
titanic['Age'].fillna(titanic['Age'].median(), inplace=True)
titanic.dropna(subset=['Embarked'], inplace=True)

# Selecionar features e variável alvo
X = titanic[['Pclass', 'Age', 'Fare', 'Sex', 'Embarked']]
y = titanic['Survived']

# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Construção da Rede Neural (Modelo Inicial)
model_titanic = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilar o modelo
model_titanic.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo inicial
history_titanic = model_titanic.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Avaliar o modelo inicial
loss, accuracy = model_titanic.evaluate(X_test, y_test)
print(f"Acurácia (Titanic - Modelo Inicial): {accuracy:.2f}")

# Função para construir o modelo com Keras Tuner
def build_model(hp):
    model = Sequential([
        Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32),
              input_dim=X_train.shape[1], activation='relu'),
        Dropout(rate=hp.Float('dropout', min_value=0.2, max_value=0.4, step=0.1)),
        Dense(units=hp.Int('units_2', min_value=16, max_value=64, step=16), activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    optimizer = Adam(learning_rate=hp.Float('learning_rate', min_value=0.001, max_value=0.1, sampling='log'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Criar um tuner e iniciar a busca por hiperparâmetros
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='my_dir',
    project_name='titanic_rna'
)

# Iniciar a busca
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)

# Obter o melhor modelo e avaliá-lo
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

# Obter o valor otimizado de 'epochs'
best_epochs = best_hps.get('epochs')

# Treinar o melhor modelo
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

# Avaliar o modelo otimizado
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Acurácia (Titanic - Modelo Otimizado): {accuracy:.2f}")

# Visualizar a evolução do treinamento
def plot_history(history):
    plt.plot(history.history['accuracy'], label='Acurácia Treino')
    plt.plot(history.history['val_accuracy'], label='Acurácia Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.title('Evolução da Acurácia no Treinamento')
    plt.show()

plot_history(history)


Trial 216 Complete [00h 00m 07s]
val_accuracy: 0.8159999847412109

Best val_accuracy So Far: 0.8479999899864197
Total elapsed time: 00h 18m 12s

Search: Running Trial #217

Value             |Best Value So Far |Hyperparameter
96                |128               |units_1
0.2               |0.3               |dropout
48                |64                |units_2
0.0048883         |0.025399          |learning_rate
17                |50                |tuner/epochs
0                 |17                |tuner/initial_epoch
2                 |4                 |tuner/bracket
0                 |3                 |tuner/round

Epoch 1/17
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6820 - loss: 0.6123 - val_accuracy: 0.7920 - val_loss: 0.4899
Epoch 2/17
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8070 - loss: 0.4444 - val_accuracy: 0.7760 - val_loss: 0.5058
Epoch 3/17
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7992 - loss: 0.4638 - val_accuracy: 0.8080 - val_loss: 

KeyboardInterrupt: 